<a href="https://colab.research.google.com/github/Vedmani/wear-particle-classification/blob/main/VGG16_with_avgpooling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install wandb -q
!wandb login 

     |████████████████████████████████| 1.9 MB 6.5 MB/s 
     |████████████████████████████████| 182 kB 66.5 MB/s 
     |████████████████████████████████| 168 kB 65.6 MB/s 
     |████████████████████████████████| 62 kB 1.4 MB/s 
     |████████████████████████████████| 168 kB 71.8 MB/s 
     |████████████████████████████████| 166 kB 68.7 MB/s 
     |████████████████████████████████| 166 kB 72.4 MB/s 
     |████████████████████████████████| 162 kB 69.1 MB/s 
     |████████████████████████████████| 162 kB 67.1 MB/s 
     |████████████████████████████████| 158 kB 68.6 MB/s 
     |████████████████████████████████| 157 kB 60.7 MB/s 
     |████████████████████████████████| 157 kB 72.4 MB/s 
     |████████████████████████████████| 157 kB 70.4 MB/s 
     |████████████████████████████████| 157 kB 59.9 MB/s 
     |████████████████████████████████| 157 kB 69.7 MB/s 
     |████████████████████████████████| 157 kB 67.8 MB/s 
     |████████████████████████████████| 157 kB 78.6 MB/s 
     |███████████

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import wandb
from wandb.keras import WandbCallback

In [4]:
tf.keras.utils.set_random_seed(1)
tf.config.experimental.enable_op_determinism()

In [5]:
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
vgg16 = VGG16(weights='imagenet',include_top=False,input_shape=(224,224,3))

58889256/58889256 [==============================] - 0s 0us/step


In [6]:
PATH = '/content/drive/MyDrive/Research Internship/final data/wear particles/Train'
test_path = '/content/drive/MyDrive/Research Internship/final data/wear particles/Test'
batch_size = 32
img_height = 224
img_width = 224

In [7]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="training",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="validation",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

test_ds = tf.keras.utils.image_dataset_from_directory(
  test_path,
  seed = 42,
  image_size=(img_height, img_width))

Found 879 files belonging to 5 classes.
Using 704 files for training.
Found 879 files belonging to 5 classes.
Using 175 files for validation.
Found 100 files belonging to 5 classes.


In [8]:
class_names = train_ds.class_names
test_class_names = test_ds.class_names
class_names == test_class_names
print(class_names)
num_classes = len(class_names)

['cutting wear', 'normal rubbing and cutting wear combined', 'normal rubbing wear', 'red oxide', 'severe sliding wear']


In [9]:
model_vgg16 = Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  vgg16,
  layers.GlobalAveragePooling2D(),
  layers.Dropout(0.2), 
  layers.Dense(num_classes, activation='softmax')
])
vgg16.trainable=False
model_vgg16.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 224, 224, 3)       0         
                                                                 
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 global_average_pooling2d (G  (None, 512)              0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 5)                 2565      
                                                                 
Total params: 14,717,253
Trainable params: 2,565
Non-trainable params: 14,714,688
________________________________________

In [10]:
model_vgg16.compile(
              optimizer= tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name='Adam'),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy']
              )

In [11]:
wandb.init(
      # Set the project where this run will be logged
      project="Transfer_Learning_New", 
      name = "vgg16_with_avgpooling",
)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: vedmanivaidya. Use `wandb login --relogin` to force relogin


In [12]:
%%time
epochs=40
history = model_vgg16.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
callbacks=[WandbCallback()]
)

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/40
22/22 [==============================] - ETA: 0s - loss: 1.4234 - accuracy: 0.3949

wandb: Adding directory to artifact (/content/wandb/run-20221128_081300-31bx9a5s/files/model-best)... Done. 0.6s


22/22 [==============================] - 122s 4s/step - loss: 1.4234 - accuracy: 0.3949 - val_loss: 1.2982 - val_accuracy: 0.5029
Epoch 2/40
22/22 [==============================] - ETA: 0s - loss: 1.2191 - accuracy: 0.5838

wandb: Adding directory to artifact (/content/wandb/run-20221128_081300-31bx9a5s/files/model-best)... Done. 0.5s


...

22/22 [==============================] - 11s 490ms/step - loss: 0.2927 - accuracy: 0.9276 - val_loss: 0.3473 - val_accuracy: 0.9029
Epoch 39/40
22/22 [==============================] - ETA: 0s - loss: 0.2922 - accuracy: 0.9233

wandb: Adding directory to artifact (/content/wandb/run-20221128_081300-31bx9a5s/files/model-best)... Done. 0.5s


22/22 [==============================] - 11s 502ms/step - loss: 0.2922 - accuracy: 0.9233 - val_loss: 0.3455 - val_accuracy: 0.9029
Epoch 40/40
22/22 [==============================] - ETA: 0s - loss: 0.2816 - accuracy: 0.9375

wandb: Adding directory to artifact (/content/wandb/run-20221128_081300-31bx9a5s/files/model-best)... Done. 0.5s


22/22 [==============================] - 11s 498ms/step - loss: 0.2816 - accuracy: 0.9375 - val_loss: 0.3421 - val_accuracy: 0.8971
CPU times: user 3min 15s, sys: 36.8 s, total: 3min 52s
Wall time: 12min 44s


In [14]:
wandb.alert(
    title = "Run finished",
    text = "Run finished after 5 epochs"
)

In [15]:
wandb.finish()

accuracy,▁▃▄▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇█████████████████
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▇▆▅▅▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▃▃▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇█▇█▇█████████████████
val_loss,█▇▆▅▅▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,0.9375
best_epoch,39
best_val_loss,0.34213
epoch,39
loss,0.28164
val_accuracy,0.89714


In [ ]:
!nvidia-smi --query-gpu=gpu_name --format=csv

name
Tesla T4
